## Install and Import Dependencies and Libraries

In [38]:
!pip3 install torch torchvision torchaudio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import requests
import torch
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

##  Instantiate Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Calculating Sentiment

In [43]:
inputs = tokenizer("The food tastes very good", return_tensors="pt")

In [44]:
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'5 stars'

## Collecting Reviews using BeautifulSoup

In [45]:
url = 'https://www.yelp.com/biz/social-brew-cafe-pyrmont'
response = requests.get(url)
lst=[]
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    regex = re.compile('.*comment.*')
    results = soup.find_all('p', {'class':regex})
    for review in reviews:
        lst.append(review.get_text())
else:
    print('Failed to retrieve the webpage. Status code:', response.status_code)


In [47]:
lst[0]

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

## Load Reviews into DataFrame and Score


In [48]:
df = pd.DataFrame(np.array(lst),columns=['reviews'])

In [24]:
def analyze_sentiment(review):
    inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs).logits
    predicted_label = torch.argmax(outputs, dim=1).item()
    result=model.config.id2label[predicted_label]
    
    return result

In [25]:
df['sentiment'] = df['reviews'].apply(analyze_sentiment)

print(df)

                                             reviews sentiment
0  Very cute coffee shop and restaurant. They hav...   5 stars
1  Six of us met here for breakfast before our wa...   4 stars
2  Great place with delicious food and friendly s...   5 stars
3  Great service, lovely location, and really ama...   5 stars
4  Great food amazing coffee and tea. Short walk ...   5 stars
5  Ricotta hot cakes! These were so yummy. I ate ...   5 stars
6  It was ok. Had coffee with my friends. I'm new...   3 stars
7  We came for brunch twice in our week-long visi...   5 stars
8  I came to Social brew cafe for brunch while ex...   5 stars
9  Great staff and food.  Must try is the pan fri...   5 stars


## The End